In [1]:
import keras
from keras.models import load_model
import numpy as np

my_ResNet50_model = load_model('/dbfs/mnt/dogImages/saved_model/dog_resnet50/data/model.h5')
dog_names = np.load('/dbfs/mnt/dogImages/bottleneck_features/DogNames.npy')

In [2]:
dog_names = [name.split('.')[1].replace("_", " ").upper() for name in dog_names]

In [3]:
dog_names = ['AFFENPINSCHER',
 'AFGHAN HOUND',
 'AIREDALE TERRIER',
 'AKITA',
 'ALASKAN MALAMUTE',
 'AMERICAN ESKIMO DOG',
 'AMERICAN FOXHOUND',
 'AMERICAN STAFFORDSHIRE TERRIER',
 'AMERICAN WATER SPANIEL',
 'ANATOLIAN SHEPHERD DOG',
 'AUSTRALIAN CATTLE DOG',
 'AUSTRALIAN SHEPHERD',
 'AUSTRALIAN TERRIER',
 'BASENJI',
 'BASSET HOUND',
 'BEAGLE',
 'BEARDED COLLIE',
 'BEAUCERON',
 'BEDLINGTON TERRIER',
 'BELGIAN MALINOIS',
 'BELGIAN SHEEPDOG',
 'BELGIAN TERVUREN',
 'BERNESE MOUNTAIN DOG',
 'BICHON FRISE',
 'BLACK AND TAN COONHOUND',
 'BLACK RUSSIAN TERRIER',
 'BLOODHOUND',
 'BLUETICK COONHOUND',
 'BORDER COLLIE',
 'BORDER TERRIER',
 'BORZOI',
 'BOSTON TERRIER',
 'BOUVIER DES FLANDRES',
 'BOXER',
 'BOYKIN SPANIEL',
 'BRIARD',
 'BRITTANY',
 'BRUSSELS GRIFFON',
 'BULL TERRIER',
 'BULLDOG',
 'BULLMASTIFF',
 'CAIRN TERRIER',
 'CANAAN DOG',
 'CANE CORSO',
 'CARDIGAN WELSH CORGI',
 'CAVALIER KING CHARLES SPANIEL',
 'CHESAPEAKE BAY RETRIEVER',
 'CHIHUAHUA',
 'CHINESE CRESTED',
 'CHINESE SHAR-PEI',
 'CHOW CHOW',
 'CLUMBER SPANIEL',
 'COCKER SPANIEL',
 'COLLIE',
 'CURLY-COATED RETRIEVER',
 'DACHSHUND',
 'DALMATIAN',
 'DANDIE DINMONT TERRIER',
 'DOBERMAN PINSCHER',
 'DOGUE DE BORDEAUX',
 'ENGLISH COCKER SPANIEL',
 'ENGLISH SETTER',
 'ENGLISH SPRINGER SPANIEL',
 'ENGLISH TOY SPANIEL',
 'ENTLEBUCHER MOUNTAIN DOG',
 'FIELD SPANIEL',
 'FINNISH SPITZ',
 'FLAT-COATED RETRIEVER',
 'FRENCH BULLDOG',
 'GERMAN PINSCHER',
 'GERMAN SHEPHERD DOG',
 'GERMAN SHORTHAIRED POINTER',
 'GERMAN WIREHAIRED POINTER',
 'GIANT SCHNAUZER',
 'GLEN OF IMAAL TERRIER',
 'GOLDEN RETRIEVER',
 'GORDON SETTER',
 'GREAT DANE',
 'GREAT PYRENEES',
 'GREATER SWISS MOUNTAIN DOG',
 'GREYHOUND',
 'HAVANESE',
 'IBIZAN HOUND',
 'ICELANDIC SHEEPDOG',
 'IRISH RED AND WHITE SETTER',
 'IRISH SETTER',
 'IRISH TERRIER',
 'IRISH WATER SPANIEL',
 'IRISH WOLFHOUND',
 'ITALIAN GREYHOUND',
 'JAPANESE CHIN',
 'KEESHOND',
 'KERRY BLUE TERRIER',
 'KOMONDOR',
 'KUVASZ',
 'LABRADOR RETRIEVER',
 'LAKELAND TERRIER',
 'LEONBERGER',
 'LHASA APSO',
 'LOWCHEN',
 'MALTESE',
 'MANCHESTER TERRIER',
 'MASTIFF',
 'MINIATURE SCHNAUZER',
 'NEAPOLITAN MASTIFF',
 'NEWFOUNDLAND',
 'NORFOLK TERRIER',
 'NORWEGIAN BUHUND',
 'NORWEGIAN ELKHOUND',
 'NORWEGIAN LUNDEHUND',
 'NORWICH TERRIER',
 'NOVA SCOTIA DUCK TOLLING RETRIEVER',
 'OLD ENGLISH SHEEPDOG',
 'OTTERHOUND',
 'PAPILLON',
 'PARSON RUSSELL TERRIER',
 'PEKINGESE',
 'PEMBROKE WELSH CORGI',
 'PETIT BASSET GRIFFON VENDEEN',
 'PHARAOH HOUND',
 'PLOTT',
 'POINTER',
 'POMERANIAN',
 'POODLE',
 'PORTUGUESE WATER DOG',
 'SAINT BERNARD',
 'SILKY TERRIER',
 'SMOOTH FOX TERRIER',
 'TIBETAN MASTIFF',
 'WELSH SPRINGER SPANIEL',
 'WIREHAIRED POINTING GRIFFON',
 'XOLOITZCUINTLI',
 'YORKSHIRE TERRIER',
 'SHIBA INU']

In [4]:
def extract_Resnet50(tensor):
	from keras.applications.resnet50 import ResNet50, preprocess_input
	return ResNet50(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

from keras.preprocessing import image                  
import numpy as np

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [5]:
def my_ResNet50_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    
    # obtain predicted vector
    predicted_vector = my_ResNet50_model.predict(bottleneck_feature)
    
    # get index with highest probability and predicted dog breed
    predicted_index= np.argmax(predicted_vector)
    predicted_breed= dog_names[predicted_index]
    
    return predicted_breed

In [6]:
my_ResNet50_predict_breed('/dbfs/mnt/dogImages/dogImages/test/134.Shiba_inu/Shiba_inu_42.jfif')

In [7]:
test_Resnet50 = np.load('/dbfs/mnt/dogImages/bottleneck_features/DogResnet50DataTest.npy')
test_targets = np.load('/dbfs/mnt/dogImages/bottleneck_features/TestTargets.npy')

In [8]:
import mlflow
import mlflow.keras

with mlflow.start_run():
  my_ResNet50_predictions = [np.argmax(my_ResNet50_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Resnet50]
  # report test accuracy
  test_accuracy = 100*np.sum(np.array(my_ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(my_ResNet50_predictions)
  mlflow.log_metric("Test_accuracy", test_accuracy)
  # log model
  mlflow.keras.log_model(my_ResNet50_model, "models")

In [9]:
import azureml
from azureml.core import Workspace

workspace_name = "AML_workspace"
workspace_location="East US"
resource_group = "Dev_RG"
subscription_id = "fabc7377-e808-46f3-bf02-1dc62b5ee1c4"

workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

/databricks/python/lib/python3.7/site-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
 category=DeprecationWarning)
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A2XSAGPAM to authenticate.
Interactive authentication successfully completed.

In [10]:
from azureml.core.compute import AksCompute, ComputeTarget
# Set the resource group that contains the AKS cluster and the cluster name
resource_group = 'Dev_RG'
cluster_name = 'AKSDev'

# Attach the cluster to your workgroup. If the cluster has less than 12 virtual CPUs, use the following instead:
attach_config = AksCompute.attach_configuration(resource_group = resource_group,
                                         cluster_name = cluster_name,
                                         cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)

aks_target = ComputeTarget.attach(workspace, 'myaks', attach_config)

In [11]:
aks_target

Out[4]: AksCompute(workspace=Workspace.create(name='AML_workspace', subscription_id='fabc7377-e808-46f3-bf02-1dc62b5ee1c4', resource_group='Dev_RG'), name=myaks, id=/subscriptions/fabc7377-e808-46f3-bf02-1dc62b5ee1c4/resourceGroups/Dev_RG/providers/Microsoft.MachineLearningServices/workspaces/AML_workspace/computes/myaks, type=AKS, provisioning_state=Creating, location=eastus, tags=None)

In [12]:
run_id1 = "15020934c28b48199012f152bd171407"
model_uri = "runs:/" + run_id1 + "/models"

In [13]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name="dogmodel",
                                                      image_name="dogdockerimage",
                                                      description="Resnet50 Dog Classification image",
                                                      synchronous=False)

In [14]:
model_image.wait_for_creation(show_output=True)

In [15]:
import requests
import json
img = open('/dbfs/mnt/cifar-100-python/Shiba_inu_54.jpg', 'rb').read()
headers = {'Content-Type':'application/json'}

resp = requests.post('http://bb6c6343-69f5-4e8e-a8b0-6b8b22533fdb.eastus.azurecontainer.io/score',data=img, headers=headers, timeout=300)

json.loads(resp.json())['dog_breed']

Out[11]: 'SHIBA INU'

In [16]:
resp

Out[12]: <Response [200]>

In [17]:
img

Out[13]: b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x07\x05\x05\x06\x05\x04\x07\x06\x05\x06\x08\x07\x07\x08\n\x11\x0b\n\t\t\n\x15\x0f\x10\x0c\x11\x18\x15\x1a\x19\x18\x15\x18\x17\x1b\x1e\'!\x1b\x1d%\x1d\x17\x18"."%()+,+\x1a /3/*2\'*+*\xff\xdb\x00C\x01\x07\x08\x08\n\t\n\x14\x0b\x0b\x14*\x1c\x18\x1c**************************************************\xff\xc0\x00\x11\x08\x01:\x01c\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xc4\x00\x1f\x01\x00\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x11\x00\x02\x01\x02\x04\x04\x03\x04\x07\x05\x04\x04\x00\x01\x02w\x00\x01\x02\x03\x11\x04\x05!1\x06\x12AQ\x07aq\x13"2\x81\x08\x14B\x91\xa1\xb1\xc1\t#3R\xf0\x15br\xd1\n\x16$4\xe1%\xf1\x17\x18\x19\x1a&\'()*56789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xda\x00\x0c\x03\x01\x00\x02\x11\x03\x11\x00?\x00\xfaF\xb9\xeds\xc6\xba/\x87u+{-V\xe3\xc9\x92\xe3\xee1\x1c~u\xd0\xd7\x98\xfce\xd0-\xf5].\xd6g\x8f\xf7\x88\xe4n\x1d\xaa\'\'\x18\xdc\xa8\xa4\xdd\x99\xe90\\Cu\x02\xcdm"\xcb\x1b\x8c\xab\xa9\xc85-|\xbd\xe0\xff\x00\x88Z\xb7\x80uo\xb0\xdd;\xdd\xe9\xa4\xf3\x13\x1c\x95\x1e\xa2\xbe\x86\xf0\xe7\x8b4\x9f\x14Y\x8b\x8d&\xe5d\xe3\xe6\x8c\x9f\x99iB\xa2\x92*T\xdcM\xba(\xa2\xb43\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n\xe4~$\xc5\xbb\xc2\x13O\x8c\x98Hj\xeb\xab\x9f\xf1\xbc>w\x83u\x04\x1d\xe3\xe9S/\x85\x8e:\xb4|\xcb\xe2\xc4\xf3\x92;\x98W\x8ca\x80\x15\x91\xa0kW\xda-\xea\xddiW2[\xcc\xa7 \x03\xf2\x9a\xe85\t\xad\xc5\x9b$\x9f\x7f\x91\x8c\xd7\x18\xb1L\xf7\x84@\x0b`\xfe\x15\xc7\x1dQ\xdf\xcd\xcb\xa3\xd4\xfa+\xc2\xbf\x17g\xbb\xb1H\xb5k2\'Q\xcc\xa3\xa3{\xd6\xac\xbf\x14\x1c\xb9[K\x15|\x7fy\xb1^#\xa4Cw\r\xb9\xf3\xd9\xb2z`\xf1Wd\xba\x9e\x06\x01r}Nk\tb*\'d\xca\x8d\x08KS\xd8#\xf8\xa1(\xcf\xda4\xf0\x07\xfb\x0cMm\xe9\xdf\x10\xf4K\xfd\xaa\xf2\xbd\xbb\x9e\xa2A\x80\r|\xf1s\xad\xcf\x1f\xfa\xb9\n\x9a\xaa5[\x87]\xd2\x1c\x9e\xb5Q\xafU+\x89\xe1\xe9\x9fW\xc7\xab\xe9\xf2\x80c\xbc\x85\xb3\xe8\xe2\xad#\xac\x8b\xba6\x0c=G5\xf2\xe5\x8f\x88%\x8dAbW\xd3\x15\xd1i\xfe5\xbf\xb0`\xf6\xf7r|\xbc\x95bJ\xd6\x8b\x19gi#9at\xbcY\xf4 \xa2\xbc\x9e\xcf\xe3\xa6\x93\r\xa1\x1a\xa4l\'Q\xfc\x1cn5\xcdj\xdf\x1e\xb5=FG\x87\xc3\xbax\xb7^\x9ed\xe3?\x8du:\xb1\xb5\xceuJm\x9e\xf5$\xa9\x12\x96\x95\xc2(\xeeN)\xb0\xdcCp\t\x82U\x90\x0e\xbbNk\xe5[\xbd_]\xd6\xa6g\xbf\xd6\xa7\xf9\x8eJ\xa3\x90\xbf\x95Z\xf0U\xee\xaf\xa4\xfcA\xd3a\xb7\xd4\xe7\x96\xdaw\xc3\xc4\xeeH5\n\xban\xc5\xfb\x1d/s\xea^\xf4R\x0ep}\xa9k\xa0\xe7\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n(\xa2\x80\n\xe5~$]\xfd\x8b\xc0\x1a\x9c\xaa\xdbXG\xf2\xf3\xd4\xe6\xba\xaa\xf2\x0f\x8cz\xac\x92\xdf\xda\xe9\x01\xb1\x02/\x9b(\xfe\xf05\x13\xf8YQ\xdc\xf1\x88\xec.\xf5y\x97\xcd\x05\x13\xae\xea\xd9K+M>5U\x00\xc9\x8f\xbcjH\xefPa\x10\x80\x07\x1cT7\xb2.\xe0\xcb\x8c\x0

In [18]:
json.loads(resp.json())['dog_breed']

--------------------------------------------------------------------------- 
 JSONDecodeError Traceback (most recent call last)
 <command-3926515677659102> in <module> 
 ----> 1 json . loads ( resp . json ( ) ) [ 'dog_breed' ] 

 /databricks/python/lib/python3.7/site-packages/requests/models.py in json (self, **kwargs) 
 895 # used. 
 896 pass 
 --> 897 return complexjson . loads ( self . text , ** kwargs ) 
 898 
 899 @ property

 /databricks/python/lib/python3.7/site-packages/simplejson/__init__.py in loads (s, encoding, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, use_decimal, **kw) 
 516 parse_constant is None and object_pairs_hook is None 
 517 and not use_decimal and not kw):
 --> 518 return _default_decoder . decode ( s ) 
 519 if cls is None : 
 520 cls = JSONDecoder

 /databricks/python/lib/python3.7/site-packages/simplejson/decoder.py in decode (self, s, _w, _PY3) 
 368 if _PY3 and isinstance ( s , bytes ) : 
 369 s = str ( s , self . encoding ) 
 --> 370 obj , end = self . raw_decode ( s ) 
 371 end = _w ( s , end ) . end ( ) 
 372 if end != len ( s ) : 

 /databricks/python/lib/python3.7/site-packages/simplejson/decoder.py in raw_decode (self, s, idx, _w, _PY3) 
 398 elif ord0 == 0xef and s [ idx : idx + 3 ] == '\xef\xbb\xbf' : 
 399 idx += 3 
 --> 400 return self . scan_once ( s , idx = _w ( s , idx ) . end ( ) ) 

 JSONDecodeError : Expecting value: line 1 column 1 (char 0)

In [19]:
import mlflow.keras
# save the mode to local or remote accessible path on the S3 or Azure Blob
mlflow.keras.save_model(my_ResNet50_model, '/dbfs/mnt/dogImages/saved_model/notop_resnet50')